In [82]:
import requests

r = requests.get('https://www.eia.gov/electricity/state/index.php')

In [83]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   State Electricity Profiles - Energy Information Administration
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="en-us" http-equiv="Content-Language"/>
  <meta content="all" name="robots"/>
  <meta content="EIA - Energy Information Administration" name="agency"/>
  <meta content="official energy statistics, data, analysis and forecasting" name="subject"/>
  <meta content="Energy Information Administration - EIA - Official Energy Statistics from the U.S. Government" name="Description"/>
  <link href="/global/styles/screen.css" media="screen, projection" rel="stylesheet" type="text/css"/>
  <link href="/global/styles/eia-styles.min.css?v=1.1.0" media="screen, projection" rel="

In [84]:
print('Number of a tags: ' + str(len(soup.find_all('a'))) + '\n------------------------------------------\n')

for i in soup.find_all('a'):
    print(i.get('href'))


Number of a tags: 282
------------------------------------------

https://twitter.com/EIAgov
None
https://www.facebook.com/eiagov
None
/
#
#menu
#menu
#menu
#menu
#menu
#menu
#menu
#nav-sources
#nav-topics
#nav-geography
#nav-tools
#nav-learn
#nav-news
/petroleum/
/petroleum/weekly/
/petroleum/supply/weekly/
/naturalgas/
http://ir.eia.gov/ngs/ngs.html
/naturalgas/weekly/
/electricity/
/electricity/monthly/
/electricity/data/browser/
/consumption/
/consumption/residential/
/consumption/commercial/
/coal/
/coal/production/quarterly/
/beta/coal/data/browser/
/renewable/
/renewable/afv/
/nuclear/
/nuclear/outages/
/totalenergy/
/totalenergy/data/monthly/
/totalenergy/data/annual/
/analysis/
/outlooks/steo/
/outlooks/aeo/
/outlooks/ieo/
/finance/
/finance/data.php
/finance/markets/crudeoil/
/environment/
/environment/emissions/carbon/
/environment/emissions/state/analysis/
/state/
/state/seds/
/maps/
/state/maps.php
/special/gulf_of_mexico/
/beta/international/
/beta/international/data/brow

In [85]:
base_url = 'https://www.eia.gov/electricity'

data = {}  #dictionary which will contain the state name and corresponding table download link
for i in soup.find_all('a'):
    url = i.get('href')
    if url and '../state/' in url and '../state/districtofcolumbia' not in url and '../state/unitedstates' not in url:
        new_url = url.replace('..', '') + '/state_tables.php'  #remove the '..' from the string
        state_name = url.replace('../state/', '')
        print('Downloading data for the state: ' + state_name)  # show the progress while running
        table_url = base_url + new_url
        r_table = requests.get(table_url)
        soup_table = BeautifulSoup(r_table.text, 'html.parser')
        for i in soup_table.find_all('a'):
            table_download_link = table_url.replace('state_tables.php', '') + str(i.get('href'))
            data[state_name] = table_download_link 


print('Successfully got data for the states: ' + str(len(data)))


Successfully got data for the states: 50


In [88]:
for key in data.keys():
    print(key + ': ' +  data[key])


alabama: https://www.eia.gov/electricity/state/alabama/xls/al.xlsx
alaska: https://www.eia.gov/electricity/state/alaska/xls/ak.xlsx
arizona: https://www.eia.gov/electricity/state/arizona/xls/az.xlsx
arkansas: https://www.eia.gov/electricity/state/arkansas/xls/ar.xlsx
california: https://www.eia.gov/electricity/state/california/xls/ca.xlsx
colorado: https://www.eia.gov/electricity/state/colorado/xls/co.xlsx
connecticut: https://www.eia.gov/electricity/state/connecticut/xls/ct.xlsx
delaware: https://www.eia.gov/electricity/state/delaware/xls/de.xlsx
florida: https://www.eia.gov/electricity/state/florida/xls/fl.xlsx
georgia: https://www.eia.gov/electricity/state/georgia/xls/ga.xlsx
hawaii: https://www.eia.gov/electricity/state/hawaii/xls/hi.xlsx
idaho: https://www.eia.gov/electricity/state/idaho/xls/id.xlsx
illinois: https://www.eia.gov/electricity/state/illinois/xls/il.xlsx
indiana: https://www.eia.gov/electricity/state/indiana/xls/in.xlsx
iowa: https://www.eia.gov/electricity/state/iowa

In [89]:
import pandas as pd

loss_data = {}  #create a dictionary where distribution loss data will be stored

for state_name in data.keys():
    print('Downloading data for: ' + state_name)  # show the progress while running
    df = pd.read_excel(io = data[state_name],
                       sheet_name= '10. Source-Disposition', header= 0,
                       skiprows = [0, 1, 2], index_col = 0, usecols = [0, 1])

    loss = df.loc["Estimated losses"]*100/(df.loc["Total disposition"] - df.loc["Direct use"])  #calculate distribution loss

    loss_data[state_name] = loss.values[0]

print('Successfully got data for the states: ' + str(len(loss_data)))

Successfully got data for the states: 50


In [81]:
##save the data in .csv file

import csv

with open('Electricity distribution loss.csv','w') as f:
    w = csv.writer(f)
    w.writerows(loss_data.items())